In [ ]:
# Authenticate user
import spotipy
import spotipy.util as util

def get_token(scope):
    token = util.prompt_for_user_token(
            username='1212629687',
            scope=scope,
            client_id='e74c52988f6d4bcebb36970a423d348d',
            client_secret='0edc87deae1a4611a97b6cebef262136',
            redirect_uri='http://localhost:8888/callback')
    return token
token = get_token(scope='user-top-read')
if token:
    sp = spotipy.Spotify(auth=token)

In [ ]:
# Experiment
# Get user's top artists
# Part 1: Function
def get_users_top_artists(limit=20):
    results = sp.current_user_top_artists(limit=limit, offset=0, time_range='medium_term')
    return results['items']

In [155]:
# Part 2: Print
for artist in get_users_top_artists():
    print artist['name']

The Go! Team
Starcadian
Daft Punk
Gramatik
Ulises Conti
Pretty Lights
The New Division
Vulfpeck
Battle Tapes
Moby
Big Black Delta
The Polish Ambassador
Death In Vegas
Lone
Low Motion Disco
Nils Frahm
18 Carat Affair
Cash+David
The La's
Beat Connection


In [158]:
# Experiment
# Part 1: Get related artists
def get_related_artists(artist):
    return sp.artist_related_artists(artist[u'id'])

In [157]:
related_artists = {}
for artist in get_users_top_artists():
    related_artists[artist['name']] = get_related_artists(artist)

In [159]:
# Part 2: Print artist and similar artists
for key, val in related_artists.iteritems():
    artist_name = key
    similar_artists = val['artists']
    similar_artist_names = [artist['name'] for artist in similar_artists]
    print artist_name
    print similar_artist_names

Daft Punk
[u'Justice', u'Cassius', u'The Chemical Brothers', u'Basement Jaxx', u'Kavinsky', u'Calvin Harris', u'Vitalic', u'MSTRKRFT', u'David Guetta', u'Gorillaz', u'Mr. Oizo', u'Digitalism', u'The Bloody Beetroots', u'Mylo', u'Moby', u'Soulwax', u'deadmau5', u'R\xf6yksopp', u'The Prodigy', u'Fatboy Slim']
Big Black Delta
[u'Austra', u'White Sea', u'Class Actress', u'Young Galaxy', u'Superhumanoids', u'MNDR', u'WIN WIN', u'Twin Shadow', u'Glasser', u'Exitmusic', u'Fol Chen', u'Zambri', u'Chromatics', u'Born Gold', u'Gardens & Villa', u'Active Child', u'Young Magic', u'Small Black', u'Hooray For Earth', u'I Break Horses']
Beat Connection
[u'Chad Valley', u'Kisses', u'Brothertiger', u'Lemonade', u'Millionyoung', u'Work Drugs', u'Tanlines', u'Summer Heart', u'Teen Daze', u'Blackbird Blackbird', u'Small Black', u'Javelin', u'Craft Spells', u'Porcelain Raft', u'Keep Shelly In Athens', u'Memory Tapes', u'Body Language', u'Clive Tanaka y su orquesta', u'Hooray For Earth', u'Neon Indian']
The

In [149]:
# Experiment
# Part 1: Get artist remixes
def get_remixes(artist, limit=50, offset=0):
    # Returns the query searching for artist + remix
    name = artist['name']
    
    def search_spotify_tracks(query):
        results = sp.search(q=query, type='track', limit=limit, offset=offset)
        tracks = [track for track in results['tracks']['items']]
        return tracks
     
    remix_tracks = search_spotify_tracks('{} remix'.format(name))
    return remix_tracks

def get_all_remixes(artist):
    all_remixes = []
    offset = 0
    limit = 50
    while True:
        remixes = get_remixes(artist, limit=limit, offset=offset)
        if not remixes:
            break
        all_remixes.extend(remixes)
        offset += limit
    return all_remixes

def get_tracks_remixed_by(artist):
    # filter only tracks the artist has remixed
    def artist_uris(track):
        return [a['uri'] for a in track['artists']]
    
    #remixes = [remix for remix in get_remixes(artist)
    #          if artist['uri'] not in remix['artists'][0]['uri']]
    remixes = [remix for remix in get_remixes(artist)
              if artist['name'] in remix['name']]
    return remixes
    
def get_tracks_others_remixed(artist):
    # filter tracks by the artist others have remixed 
    def artist_uris(track):
        return [a['uri'] for a in track['artists']]
    
    remixes = [remix for remix in get_remixes(artist)
               if artist['uri'] in remix['artists'][0]['uri']]
    return remixes

In [ ]:
# Part 2: Print artist remixes 
# TODO: Add boards of canada test, should find 4
for artist in get_users_top_artists():
    remixes = get_all_remixes(artist)
    print artist['name'] ,': ', [t['name'] for t in remixes]

In [137]:
# Test
# Find all remixes
boc_uri = 'spotify:artist:2VAvhf61GgLYmC6C8anyX1'
artist = sp.artist(boc_uri)
remixes = get_all_remixes(artist)
print artist['name']
print [t['name'] for t in remixes]

Boards of Canada
[u'Last walk around mirror lake - Boards of Canada remix', u'Broken Drum - Remixed By Boards Of Canada', u'Mr Mistake (Boards Of Canada Remix)', u'Dayvan Cowboy (Odd Nosdam Remix)', u'Dead Dogs Two (Boards Of Canada Remix) - mixed', u'Dead Dogs Two - Boards of Canada remix', u'Kid For Today (Stereolab Mix)', u'Dead Dogs Two (Boards Of Canada Remix) - mixed', u'Poppy Seed (Boards Of Canada Remix)', u'Dayvan Cowboy (Remix)', u'Last walk around mirror lake - Boards of Canada remix', u'Sandsings (remixed by Boards of Canada)', u'An Eagle In Your Mind (Consumed And Regurgitated By The P.B.O.Ster!)', u'Poppy Seed (Reprise) (Boards Of Canada Remix)']


In [147]:
# Test
# Find all artist tracks remixed by others
boc_uri = 'spotify:artist:4tZwfgrHOc3mvqYlEYSvVi'
artist = sp.artist(boc_uri)
remixes = get_tracks_others_remixed(artist)
print artist['name']
print [t['artists'][0]['name'] + '-' + t['name'] for t in remixes]

Daft Punk
[u'Daft Punk-Derezzed - Remixed by The Glitch Mob', u'Daft Punk-Get Lucky - Daft Punk Remix', u'Daft Punk-Aerodynamic (Daft Punk Remix)', u'Daft Punk-Fall - Remixed by M83 VS Big Black Delta', u'Daft Punk-The Grid - Remixed by The Crystal Method', u'Daft Punk-Derezzed - Remixed by Avicii', u'Daft Punk-The Son of Flynn - Remixed by Moby', u'Daft Punk-Adagio for TRON - Remixed by Teddybears', u'Daft Punk-End of Line - Remixed by Photek', u'Daft Punk-The Son of Flynn - Remixed by Ki:Theory', u'Daft Punk-Solar Sailer - Remixed by Pretty Lights', u'Daft Punk-End of Line - Remixed by Boys Noize', u'Daft Punk-Rinzler - Remixed by Kaskade', u'Daft Punk-C.L.U. - Remixed by Paul Oakenfold', u'Daft Punk-Encom Part 2 - Remixed by Com Truise', u'Daft Punk-TRON Legacy (End Titles) - Remixed by Sander Kleinenberg', u'Daft Punk-Arena - Remixed by The Japanese Popstars', u'Daft Punk-Robot Rock - Soulwax Remix', u'Daft Punk-Human After All - SebastiAn Remix', u'Daft Punk-Harder Better Faster S

In [154]:
# Test
# Find all tracks remixed by artist
boc_uri = 'spotify:artist:43mWhBXSflupNLuNjM5vff'
artist = sp.artist(boc_uri)
remixes = get_tracks_remixed_by(artist)
print artist['name']
print [t['artists'][0]['name'] + '-' + t['name'] for t in remixes]

Soulwax
[u'Tame Impala-Let It Happen - Soulwax Remix', u'Jungle-Julia (Soulwax Remix)', u"The Rolling Stones-You Can't Always Get What You Want (Soulwax Remix) - Remix", u'Daft Punk-Robot Rock - Soulwax Remix', u'Gorillaz-DARE - Soulwax Remix', u'Gorillaz-DARE - Soulwax Remix', u'Joe Goddard-Gabriel - Soulwax Remix', u'Hot Chip-Huarache Lights - Soulwax Remix', u'Muse-Muscle Museum [Soulwax Remix]', u'Justice-Phantom Pt II - Soulwax Remix', u'Hot Chip-Huarache Lights - Soulwax Remix', u'LCD Soundsystem-Get Innocuous! - Soulwax Remix', u'LCD Soundsystem-You Wanted A Hit - Soulwax Remix', u'Gorillaz-DARE - Soulwax Remix', u'Ladytron-Seventeen - Soulwax Mix', u'Hot Chip-Huarache Lights - Soulwax Remix', u'Late of the Pier-Best in The Class - Soulwax Remix', u'LCD Soundsystem-Get Innocuous! - Soulwax Remix', u'Ladytron-Seventeen - Soulwax Mix', u'Justice-Phantom Pt II - Soulwax Remix', u'Zita Swoon-My Bond With You and Your Planet: Disoc! - Soulwax Remix']


In [151]:
remix = remixes[0]

In [153]:
remix['name']

u'Daft Punk / Skrillex (Remix)'